In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/essays_amini_fatemeh/HuluMed_Project/Hulu-Med

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/essays_amini_fatemeh/HuluMed_Project/Hulu-Med


In [2]:
!pip install -q --no-cache-dir torch==2.4.0 torchvision==0.19.0 --extra-index-url https://download.pytorch.org/whl/cu118
!pip install -q --no-cache-dir bitsandbytes==0.43.2 accelerate transformers==4.51.2
!pip install -q --no-cache-dir decord imageio nibabel opencv-python pillow gdown

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, BitsAndBytesConfig

In [ ]:
#!pip install --upgrade --force-reinstall --no-cache-dir bitsandbytes==0.43.2


In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    "ZJU-AI4H/Hulu-Med-7B",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=quantization_config,
    attn_implementation="flash_attention_2"
)

processor = AutoProcessor.from_pretrained(
    "ZJU-AI4H/Hulu-Med-7B",
    trust_remote_code=True
)


print("مدل با موفقیت لود شد! حالا تست با عکس.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
from PIL import Image
import torch

def run_infer(image_path, prompt="این تصویر پزشکی را تحلیل کن.",
              max_new_tokens=512, temperature=0.6):

    # استفاده از model و processor که قبلاً لود کردی
    global model, processor

    # لود تصویر
    image = Image.open(image_path).convert("RGB")

    # آماده‌سازی ورودی
    inputs = processor(
        images=image,
        text=prompt,
        return_tensors="pt"
    ).to(model.device)

    # تولید خروجی
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature
    )

    # تبدیل به متن
    result = processor.decode(output_ids[0], skip_special_tokens=True)

    print("\n======================")
    print(result)
    print("======================\n")


In [ ]:
run_infer(
    "/content/drive/MyDrive/essays_amini_fatemeh/HuluMed_Project/notebooks/sample.jpg",
    prompt="لطفاً گزارش کامل پزشکی این تصویر MRI را بنویس. <image>",
)

